In [32]:
import requests
import json  ### Intentar cambiar el formato JSON por uno más específico de Python
import sys
from Bio impor SeqIO
from Bio.SeqRecord import SeqRecord

In [1]:
print("Hola")

Hola


- https://rest.ensembl.org: REST API de Ensembl
- overlap/id: Endpoint específico de la API (es decir, método de acceso a las variantes de un determinado gen así como a información específica del mismo a través del identificador (id)
- feature = variation --> lo que queremos obtener son las variaciones de dicho gen
- species = {especie}, en nuestro caso queremos que sea en humano, pero la función también nos sirve para otras especies

- solicitud = requests.get(rest_api+extension_variantes, headers={ "Content-Type" : "application/json"}) --> Hace una solicitud HTTP al servidor Ensembl, con la información que hemos escrito previamente a través de la biblioteca requests (sirve para esto)
  1. requests.get() --> Envía una solicitud HTTP al servidor que indiquemos a continuación
  2. headers = {"Content-Type" : "application/json"} --> es un parámetro opcional que indica que la información que devuelva la solicitud que hemos hecho se encuentre en formato JSON
 


1. "solicitud.ok" --> Esto se utiliza para saber si la solicitud ha sido exitosa y hemos obtenido respuesta por parte del servidor.
2. "solicitud.raise_for_status() --> De esta forma podemos manejar los errores que hayan podido surgir durante la misma
3. "sys.exit()" --> Se utiliza para terminar el programa en ese punto exacto, es decir, después de realizar la consulta a Ensembl


- variantes = solicitud.json() --> Esto indica que la información que ha devuelto nuestra consulta, la cual se encuentra en formato JSON, se almacene en un diccionario de Python, para poder procesar dicha información más adelante

## Aceso a todas las variantes de un gen a través de la API de Ensembl

In [33]:
def acceso_variantes_ensembl (identificador_gen, especie):
    """ 
    DESCRIPCIÓN DE LA FUNCIÓN"""
    
    rest_api = "https://rest.ensembl.org"
    extension_variantes = f"/overlap/id/{identificador_gen}?feature=variation;species={especie}"

    solicitud = requests.get(rest_api+extension_variantes, headers={"Content-Type" : "application/json"})


    if not solicitud.ok:
        solicitud.raise_for_status()
        sys.exit()

    variantes = solicitud.json()
    
    return variantes



## Obtención de las variantes patogénicas a partir de las variantes que hemos obtenido de la API de Ensembl

In [34]:
def obtener_variantes_patogenicas (identificador_gen, especie):
    """
    DESCRIPCIÓN DE LA FUNCIÓN
    """

    variantes = acceso_variantes_ensembl (identificador_gen, especie)
    variantes_patogenicas = []

    for var in variantes:
        if "clinical_significance" in var and "pathogenic" in var["clinical_significance"]:
            variantes_patogenicas.append(var)


    return variantes_patogenicas
            

## Obtención de las variantes benignas a partir de las variantes que hemos obtenido de la API de Ensembl

In [36]:
def obtener_variantes_benignas (identificador_gen, especie):
    """
    DESCRIPCIÓN DE LA FUNCIÓN
    """

    variantes = acceso_variantes_ensembl (identificador_gen, especie)
    variantes_benignas = []

    for var in variantes:
        if "clinical_significance" in var and "benign" in var["clinical_significance"]:
            variantes_benignas.append(var)


    return variantes_benignas
            

In [37]:
variantes_patogenicas = obtener_variantes_patogenicas ("ENSG00000141510", "human")

In [143]:
for i in variantes_patogenicas[:5]:
    print (i["id"])

rs1555523630
rs730882017
rs2150988776
rs876658982
rs587781664


## Obtención de la secuencia de las variantes, tanto patogénicas, como benignas

Como al utilizar la API de Ensembl para obtener las variantes no obtenemos la secuencia completa del gen con estas variaciones, sino que solo obtenemos los nucleótidos que han sufrido la variación. Para obtener dichas secuencias con las variaciones es necesario seguir dos pasos:

1. Obtenemos la secuencia de referencia del gen.
   
3. Aplicamos las variaciones que hemos obtenido a través de la API sobre la secuencia de referencia del gen. 

In [77]:
def obtener_secuencia_variante (identificador_variante):

    rest_api = "https://rest.ensembl.org"
    extension_variantes = f"/variation/homo_sapiens/{identificador_variante}?content-type=application/json"

    solicitud = requests.get(rest_api+extension_variantes, headers={"Content-Type" : "application/json"})


    if not solicitud.ok:
        solicitud.raise_for_status()
        sys.exit()

    variante_informacion = solicitud.json()
    print (variante_informacion)
    
    #Ahora vamos a obtener la información relativa a la localización de la variante en la secuencia

    localizacion = variante_informacion["mappings"][0]
    cromosoma = localizacion["seq_region_name"]
    inicio_variante = localizacion["start"]
    fin_variante = localizacion["end"]

    flanco_secuencia = 100
    inicio_secuencia = inicio_variante - flanco_secuencia
    fin_secuencia = fin_variante + flanco_secuencia

    extension_secuencia_variantes = f"/sequence/region/human/{cromosoma}:{inicio_secuencia}..{fin_secuencia}:1?content-type=text/plain"
    solicitud2 = requests.get(rest_api+extension_secuencia_variantes, headers={"Content-Type": "text/plain"})

    if not solicitud2.ok:
        solicitud2.raise_for_status()
        sys.exit()

    secuencia_variante = solicitud2.text
        
    print (secuencia_variante)
    return secuencia_variante

        
    

In [79]:
obtener_secuencia_variante ("rs587781664")

{'evidence': ['Cited', 'Phenotype_or_Disease'], 'ambiguity': 'N', 'minor_allele': None, 'source': 'Variants (including SNPs and indels) imported from dbSNP', 'clinical_significance': ['not provided', 'likely pathogenic', 'pathogenic'], 'MAF': None, 'name': 'rs587781664', 'most_severe_consequence': 'splice_acceptor_variant', 'mappings': [{'allele_string': 'T/A/C/G', 'end': 7669692, 'strand': 1, 'assembly_name': 'GRCh38', 'location': '17:7669692-7669692', 'ancestral_allele': 'T', 'coord_system': 'chromosome', 'seq_region_name': '17', 'start': 7669692}], 'var_class': 'SNP', 'synonyms': ['VCV000635383', 'RCV000786810', 'VCV000141332', 'RCV000206779', 'RCV002288631', 'RCV000129814', 'VCV001264355', 'RCV000213070', 'RCV001667873', 'TP53_g.17908A>G']}
ACAAGAAGTGGAGAATGTCAGTCTGAGTCAGGCCCTTCTGTCTTGAACATGAGTTTTTTATGGCGGGAGGTAGACTGACCCTTTTTGGACTTCAGGTGGCTGTAGGAGACAGAAGCAGGGAGGAGAGATGACATCACATGAGTGAGAGGGTCTGTGCCCCTTTTCCCTGACCAATGCTTTGAAGGGCCTAAGGCTGGGACA


'ACAAGAAGTGGAGAATGTCAGTCTGAGTCAGGCCCTTCTGTCTTGAACATGAGTTTTTTATGGCGGGAGGTAGACTGACCCTTTTTGGACTTCAGGTGGCTGTAGGAGACAGAAGCAGGGAGGAGAGATGACATCACATGAGTGAGAGGGTCTGTGCCCCTTTTCCCTGACCAATGCTTTGAAGGGCCTAAGGCTGGGACA'

In [76]:

sec1 = "GGTGAGAGGCTGCCGAGGGGGAAGCAGCCCAGTCCTAGAAGCAGGGGAGGGAGAGAACCTTGTGCCAGCCGTGGCCGGAGGGGAGGAGGGACGGTTGGTTCCTGAGTTATGAATGGAGCCACCCCCTCCCTTGTTGTCATGCAGCCCGCAGACTGACCCAGTCTCCAGCCTTTGTTCCCCTCCCGGAACCCGGGCACTGGGC"
sec2 = "GGTGAGAGGCTGCCGAGGGGGAAGCAGCCCAGTCCTAGAAGCAGGGGAGGGAGAGAACCTTGTGCCAGCCGTGGCCGGAGGGGAGGAGGGACGGTTGGTTCCTGAGTTATGAATGGAGCCACCCCCTCCCTTGTTGTCATGCAGCCCGCAGACTGACCCAGTCTCCAGCCTTTGTTCCCCTCCCGGAACCCGGGCACTGGGC"
print (sec1 == sec2)

True


In [52]:
def variantes_region (identificador_gen, especie, cromosoma_gen, pos_inicio, pos_fin):
    variantes_patogenicas = obtener_variantes_patogenicas (identificador_gen, especie)
    diccionario_variantes = {}

    for i in variantes_patogenicas:
        rest_api = "https://rest.ensembl.org"
        extension_variantes = f"/variation/homo_sapiens/{identificador_variante}?content-type=application/json"

        solicitud = requests.get(rest_api+extension_variantes, headers={"Content-Type" : "application/json"})


        if not solicitud.ok:
            solicitud.raise_for_status()
            sys.exit()

        variante_informacion = solicitud.json()
    
    #Ahora vamos a obtener la información relativa a la localización de la variante en la secuencia

        localizacion = variante_informacion["mappings"][0]
        cromosoma = localizacion["seq_region_name"]
        inicio_variante = localizacion["start"]
        fin_variante = localizacion["end"]


        if cromosoma == cromosoma_gen and inicio_variante > pos_inicio and inicio_variante < pos_fin and fin_variante > pos_inicio and fin_variante < pos_fin:
            diccionario_variantes[i["id"]] = i

    return diccionario_variantes


        

    
    

In [66]:
def guardar_fasta (identificador_variante):
    x = SeqRecord(Seq((obtener_secuencia_variante(identificador_variante))))
    SeqIO.write (x, "secuencia.fasta", "fasta")
    

In [67]:
def obtener_secuencia_gen(id_gen):
    rest_api = "https://rest.ensembl.org"
    extension_gen = f"/sequence/id/{id_gen}?content-type=text/plain"
    
    solicitud = requests.get(rest_api + extension_gen, headers={"Content-Type": "text/plain"})

    if not solicitud.ok:
        solicitud.raise_for_status()
        return None

    secuencia_gen = solicitud.text

In [73]:
x = obtener_secuencia_gen ("ENSG00000141510")